In [9]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 05:11:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2022-10-27 05:17:09--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.164.55.59, 18.164.55.125, 18.164.55.165, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.164.55.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet.1’

fhvhv_tripdata_2021 100%[===================>] 294.61M   147MB/s    in 2.0s    

2022-10-27 05:17:12 (147 MB/s) - ‘fhvhv_tripdata_2021-01.parquet.1’ saved [308924937/308924937]



In [4]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [3]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [4]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [5]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('originating_base_num', 'string'),
 ('request_datetime', 'timestamp'),
 ('on_scene_datetime', 'timestamp'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'bigint'),
 ('DOLocationID', 'bigint'),
 ('trip_miles', 'double'),
 ('trip_time', 'bigint'),
 ('base_passenger_fare', 'double'),
 ('tolls', 'double'),
 ('bcf', 'double'),
 ('sales_tax', 'double'),
 ('congestion_surcharge', 'double'),
 ('airport_fee', 'double'),
 ('tips', 'double'),
 ('driver_pay', 'double'),
 ('shared_request_flag', 'string'),
 ('shared_match_flag', 'string'),
 ('access_a_ride_flag', 'string'),
 ('wav_request_flag', 'string'),
 ('wav_match_flag', 'string')]

In [26]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [27]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [28]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [31]:
df = df.repartition(24)

In [32]:
df.write.parquet('fhvhv/2021/01/')

In [33]:
!ls -lh fhvhv/2021/01/

total 226M
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc    0 Oct 27 05:43 _SUCCESS
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc 9.4M Oct 27 05:43 part-00000-77e73f70-20df-4bb0-9d53-904109e673eb-c000.snappy.parquet
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc 9.4M Oct 27 05:43 part-00001-77e73f70-20df-4bb0-9d53-904109e673eb-c000.snappy.parquet
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc 9.4M Oct 27 05:43 part-00002-77e73f70-20df-4bb0-9d53-904109e673eb-c000.snappy.parquet
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc 9.4M Oct 27 05:43 part-00003-77e73f70-20df-4bb0-9d53-904109e673eb-c000.snappy.parquet
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc 9.4M Oct 27 05:43 part-00004-77e73f70-20df-4bb0-9d53-904109e673eb-c000.snappy.parquet
-rw-r--r-- 1 dataengineeringcampjbc dataengineeringcampjbc 9.4M Oct 27 05:43 part-00005-77e73f70-20df-4bb0-9d53-904109e673eb-c000.snappy.parquet
-rw-r--r-- 1 dataengineeringcampj

In [4]:
df = spark.read.parquet('fhvhv/2021/01/')

In [5]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [12]:
df.withColumn('pickup_date',F.to_date(df.pickup_datetime)).withColumn('dropoff_date',F.to_date(df.dropoff_datetime)).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|           HV0003|              B02872|2021-01-29 18:41:29|2021-01-29 18:51:44|         256|         256|   null| 2021-01-29|  2021-01-29|
|           HV0005|              B02510|2021-01-12 18:39:57|2021-01-12 19:00:30|          61|          76|   null| 2021-01-12|  2021-01-12|
|           HV0003|              B02765|2021-01-29 19:25:19|2021-01-29 19:41:49|         113|         262|   null| 2021-01-29|  2021-01-29|
|           HV0005|              B02510|2021-01-22 20:08:47|2021-01-22 20:20:16|          22|         227|   null| 2021-01-22|  2021-01-22|
|           HV0003| 

In [8]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID').filter(df.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-29 18:41:29|2021-01-29 18:51:44|         256|         256|
|2021-01-29 19:25:19|2021-01-29 19:41:49|         113|         262|
|2021-01-29 14:13:39|2021-01-29 14:28:50|         198|          95|
|2021-01-16 10:11:49|2021-01-16 10:24:02|         235|          41|
|2021-01-11 13:04:13|2021-01-11 13:11:39|         138|         223|
|2021-01-28 21:54:17|2021-01-28 22:08:09|         155|          89|
|2021-01-04 16:43:38|2021-01-04 16:47:48|          26|         227|
|2021-01-12 04:57:34|2021-01-12 05:15:53|         221|         172|
|2021-01-25 10:11:13|2021-01-25 10:39:33|         258|         181|
|2021-01-13 21:32:29|2021-01-13 22:15:28|         140|          72|
|2021-01-20 18:16:52|2021-01-20 18:31:39|          49|         217|
|2021-01-05 16:54:13|2021-01-05 17:16:26|       

In [14]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [17]:
crazy_stuff_udf=F.udf(crazy_stuff, returnType=types.StringType())

In [20]:
df.withColumn('pickup_date',F.to_date(df.pickup_datetime)).withColumn('dropoff_date',F.to_date(df.dropoff_datetime)) \
.withColumn('base_id',crazy_stuff_udf(df.dispatching_base_num)) \
.select("base_id",'pickup_date','dropoff_date','PULocationID','DOLocationID') \
.show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b38| 2021-01-29|  2021-01-29|         256|         256|
|  e/9ce| 2021-01-12|  2021-01-12|          61|          76|
|  s/acd| 2021-01-29|  2021-01-29|         113|         262|
|  e/9ce| 2021-01-22|  2021-01-22|          22|         227|
|  s/acd| 2021-01-29|  2021-01-29|         198|          95|
|  e/b47| 2021-01-16|  2021-01-16|         235|          41|
|  e/b35| 2021-01-11|  2021-01-11|         138|         223|
|  e/b38| 2021-01-28|  2021-01-28|         155|          89|
|  e/a39| 2021-01-04|  2021-01-04|          26|         227|
|  e/a39| 2021-01-12|  2021-01-12|         221|         172|
|  e/b38| 2021-01-25|  2021-01-25|         258|         181|
|  a/b43| 2021-01-13|  2021-01-13|         140|          72|
|  s/acd| 2021-01-20|  2021-01-20|          49|         217|
|  a/b43| 2021-01-05|  2